Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import combinations
import numpy as np

## Simple Test Problem

In [6]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]

## Common tests

In [11]:
problem = np.load('lab2/problem_r1_500.npy')

In [5]:
# Negative values?
np.any(problem < 0)

False

In [6]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

True

In [7]:
# Symmetric matrix?
np.allclose(problem, problem.T)

True

In [8]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

True

In [12]:
print(problem)

[[ 0.         25.3424027  51.74329731 ... 24.97291088 31.20251812
  25.12205087]
 [39.47248084  0.         61.014369   ... 32.68309306 60.13468579
  62.83336495]
 [24.02627774 38.9698804   0.         ... 36.55178443 75.32502061
  20.73493073]
 ...
 [15.71772509 49.29464713 14.06029894 ...  0.         98.24926461
   5.17687909]
 [45.36233412 62.85710074 44.8559233  ... 54.91658175  0.
  91.99276476]
 [21.23253359 56.23088418 28.57520224 ...  7.35581983 54.86333718
   0.        ]]


In [1]:
import numpy as np
import random
import time
from math import sqrt
import os


def calculate_distance(city1, city2):
    """Calculates Euclidean distance between two cities (coordinates)."""
    return sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

def create_distance_matrix(coords):
    """Pre-calculates all city-to-city distances."""
    N = len(coords)
    matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(i + 1, N):
            dist = calculate_distance(coords[i], coords[j])
            matrix[i, j] = matrix[j, i] = dist
    return matrix

def load_and_prepare_data(filename):
    """Loads and converts data to a distance matrix if needed."""
    try:
        data = np.load(filename)
        if data.ndim == 2 and data.shape[1] >= 2:
            dist_matrix = create_distance_matrix(data)
            N_cities = dist_matrix.shape[0]
            return dist_matrix, N_cities
        elif data.ndim == 2 and data.shape[0] == data.shape[1]:
            N_cities = data.shape[0]
            return data, N_cities
        else:
            raise ValueError("Unsupported data format.")
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        return None, 0


def calculate_tour_length(tour, dist_matrix):
    """
    Calculates the total length of a given tour using NumPy vectorization.
    """
    tour_array = np.array(tour)
    current_cities = tour_array
    next_cities = np.roll(tour_array, -1) 
    
    return np.sum(dist_matrix[current_cities, next_cities])


def two_opt_swap(tour, i, j):
    """Performs the 2-opt swap: reverses the segment between i and j."""
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def two_opt_local_search(tour, dist_matrix):
    """Iteratively applies 2-opt swaps until no further improvement is found."""
    best_tour = list(tour)
    N = len(best_tour)
    
    while True:
        improved = False
        current_length = calculate_tour_length(best_tour, dist_matrix)
        
        for i in range(1, N - 1):
            for j in range(i + 1, N):
                new_tour = two_opt_swap(best_tour, i, j)
                new_length = calculate_tour_length(new_tour, dist_matrix)
                
                if new_length < current_length:
                    best_tour = new_tour
                    current_length = new_length
                    improved = True
        
        if not improved:
            break
            
    return best_tour, current_length


def initialize_population(N_cities, pop_size):
    """Creates an initial population of random tours."""
    population = []
    base_tour = list(range(N_cities))
    for _ in range(pop_size):
        shuffled_tour = list(base_tour)
        random.shuffle(shuffled_tour)
        population.append(shuffled_tour)
    return population

def select_parents(population, fitnesses, k=5):
    """Tournament Selection: Chooses the fittest individual (min length) from k random individuals."""
    tournament_indices = random.sample(range(len(population)), k)
    best_index = min(tournament_indices, key=lambda i: fitnesses[i])
    return population[best_index]

def order_crossover(parent1, parent2):
    """Order Crossover (OX)."""
    N = len(parent1)
    start, end = sorted(random.sample(range(N), 2))
    
    child = [None] * N
    child[start:end+1] = parent1[start:end+1]
    
    p2_index = (end + 1) % N
    c1_index = (end + 1) % N
    
    while None in child:
        city = parent2[p2_index]
        if city not in child:
            child[c1_index] = city
            c1_index = (c1_index + 1) % N
        p2_index = (p2_index + 1) % N
        
    return child

def swap_mutation(tour, mutation_rate=0.05):
    """Swaps two randomly selected city positions in the tour."""
    if random.random() < mutation_rate:
        N = len(tour)
        idx1, idx2 = random.sample(range(N), 2)
        tour[idx1], tour[idx2] = tour[idx2], tour[idx1]
    return tour

def solve_tsp_ga_hybrid(filename, pop_size, generations, mutation_rate, two_opt_freq=100):
    """Runs the HYBRID GA solver with NumPy speed and 2-opt quality."""
    
    dist_matrix, N_cities = load_and_prepare_data(filename)
    if N_cities == 0:
        return float('inf'), None

    if N_cities <= 50:
        generations = min(generations, 500)
    
    population = initialize_population(N_cities, pop_size)
    best_tour = None
    best_length = float('inf')
    
    for gen in range(generations):
        fitnesses = [calculate_tour_length(tour, dist_matrix) for tour in population]
        
        current_best_index = np.argmin(fitnesses)
        current_best_tour = population[current_best_index]
        current_best_length = fitnesses[current_best_index]
        
        if current_best_length < best_length:
            best_length = current_best_length
            best_tour = current_best_tour
            
        
        if gen % two_opt_freq == 0 and best_tour is not None:
            
            # Only run 2-opt on N>=500 problems when the generation count is a multiple of 500.
            if N_cities < 500 or gen % 500 == 0:
                optimized_tour, optimized_length = two_opt_local_search(best_tour, dist_matrix)
                if optimized_length < best_length:
                    best_length = optimized_length
                    best_tour = optimized_tour
                    population[0] = list(best_tour) 
        # -----------------------------------
            
        # Elitism: Keep the overall best tour for the next generation
        new_population = [list(best_tour)] 
        
        # Fill the rest of the new population
        while len(new_population) < pop_size:
            parent1 = select_parents(population, fitnesses)
            parent2 = select_parents(population, fitnesses)
            
            child = order_crossover(parent1, parent2)
            child = swap_mutation(child, mutation_rate)
            
            new_population.append(child)
            
        population = new_population

    return best_length, best_tour



def batch_solve_problems(file_list, default_params):
    """Iterates through all files and applies the GA solver."""
    all_results = {}
    
    print("\n--- Starting Hybrid GA Execution (Speed Optimized) ---")
    
    for full_path in file_list:
        
        # Get base filename and N_cities
        filename = os.path.basename(full_path)
        size_str = filename.split('_')[-1].replace('.npy', '')
        N_cities = int(size_str) if size_str.isdigit() else 0
        
        # BASE PARAMETERS
        POP_SIZE = default_params['pop_size']
        GENERATIONS = default_params['generations']
        MUTATION_RATE = default_params['mutation_rate']
        TWO_OPT_FREQ = default_params['two_opt_freq']
        
        # 🚨 SCALING FOR LARGE PROBLEMS (NEW SPEED FOCUS) 🚨
        if N_cities >= 500:
            POP_SIZE = 100 
            GENERATIONS = 1000 # Reduced total generations
            MUTATION_RATE = 0.05
            TWO_OPT_FREQ = 300 # Run 2-opt very sparsely
        elif N_cities >= 100:
            POP_SIZE = 100
            GENERATIONS = 1000
            MUTATION_RATE = 0.1
            TWO_OPT_FREQ = 20
        else: # Small problems (N < 100)
            POP_SIZE = 50
            GENERATIONS = 500
            MUTATION_RATE = 0.15
            TWO_OPT_FREQ = 5

        print(f"\nProcessing {filename} (N={N_cities}). Params: Pop={POP_SIZE}, Gen={GENERATIONS}")
        
        start_time = time.time()
        best_score, best_tour = solve_tsp_ga_hybrid(full_path, POP_SIZE, GENERATIONS, MUTATION_RATE, TWO_OPT_FREQ)
        end_time = time.time()
        
        runtime = end_time - start_time
        
        all_results[filename] = {
            'N_cities': N_cities,
            'best_score': f"{best_score:.2f}",
            'runtime_sec': f"{runtime:.2f}",
            'parameters': f"Pop={POP_SIZE}, Gen={GENERATIONS}, Mut={MUTATION_RATE}, 2opt_F={TWO_OPT_FREQ}"
        }
        
        print(f"[{filename}] Final Score: {best_score:.2f} | Time: {runtime:.2f}s")
        
    return all_results

# --- EXECUTION ---

if __name__ == '__main__':
    
    rng = np.random.default_rng(seed = 42)
    problem_folder = 'lab2' 
    
    # List of files, EXCLUDING 'test_problem.npy'
    all_files_names = [
        'problem_g_10.npy', 'problem_g_100.npy', 'problem_g_20.npy', 
        'problem_g_200.npy', 'problem_g_50.npy', 'problem_g_500.npy', 'problem_r1_10.npy', 
        'problem_r1_100.npy', 'problem_r1_20.npy', 'problem_r1_200.npy', 
        'problem_r1_50.npy', 'problem_r1_500.npy', 'problem_r2_10.npy', 'problem_r2_100.npy',
        'problem_r2_20.npy', 'problem_r2_200.npy', 'problem_r2_50.npy', 
        'problem_r2_500.npy'
    ]

    full_file_paths = [os.path.join(problem_folder, f) for f in all_files_names]

    DEFAULT_PARAMS = {
        'pop_size': 100,
        'generations': 700,
        'mutation_rate': 0.05,
        'two_opt_freq': 20
    }

    results = batch_solve_problems(full_file_paths, DEFAULT_PARAMS)

    # Print the final summary table
    print("\n\n--- RESULTS SUMMARY (Hybrid GA: NumPy + 2-opt) ---")
    print(f"{'Problem':<20} | {'N':<4} | {'Best Score':<12} | {'Time (s)':<10} | Parameters")
    print("-" * 75)
    
    # Custom sorting key (using the base filename key stored in 'results')
    def custom_sort_key_summary(filename):
        parts = filename.split('_')
        prob_type = parts[1]
        size = int(parts[-1].replace('.npy', ''))
        return (prob_type, size)
    
    sorted_results = sorted(results.items(), key=lambda item: custom_sort_key_summary(item[0]))

    for filename, data in sorted_results:
        print(f"{filename:<20} | {data['N_cities']:<4} | {data['best_score']:<12} | {data['runtime_sec']:<10} | {data['parameters']}")
    
    print("-" * 75)


--- Starting Hybrid GA Execution (Speed Optimized) ---

Processing problem_g_10.npy (N=10). Params: Pop=50, Gen=500
[problem_g_10.npy] Final Score: 1398.03 | Time: 1.55s

Processing problem_g_100.npy (N=100). Params: Pop=100, Gen=1000
[problem_g_100.npy] Final Score: 2624.30 | Time: 32.21s

Processing problem_g_20.npy (N=20). Params: Pop=50, Gen=500
[problem_g_20.npy] Final Score: 1362.06 | Time: 2.14s

Processing problem_g_200.npy (N=200). Params: Pop=100, Gen=1000
[problem_g_200.npy] Final Score: 4129.90 | Time: 121.42s

Processing problem_g_50.npy (N=50). Params: Pop=50, Gen=500
[problem_g_50.npy] Final Score: 2189.28 | Time: 7.06s

Processing problem_g_500.npy (N=500). Params: Pop=100, Gen=1000
[problem_g_500.npy] Final Score: 5807.69 | Time: 344.07s

Processing problem_r1_10.npy (N=10). Params: Pop=50, Gen=500
[problem_r1_10.npy] Final Score: 241.31 | Time: 0.99s

Processing problem_r1_100.npy (N=100). Params: Pop=100, Gen=1000
[problem_r1_100.npy] Final Score: 884.35 | Time: 21.

In [2]:
if __name__ == '__main__':
    
    rng = np.random.default_rng(seed = 42)
    problem_folder = 'lab2' 
    
    # List of files, EXCLUDING 'test_problem.npy'
    all_files_names = [
        'problem_g_1000.npy',
        'problem_r1_1000.npy',
        'problem_r2_1000.npy',
    ]

    full_file_paths = [os.path.join(problem_folder, f) for f in all_files_names]

    DEFAULT_PARAMS = {
        'pop_size': 100,
        'generations': 700,
        'mutation_rate': 0.05,
        'two_opt_freq': 20
    }

    results = batch_solve_problems(full_file_paths, DEFAULT_PARAMS)

    # Print the final summary table
    print("\n\n--- RESULTS SUMMARY (Hybrid GA: NumPy + 2-opt) ---")
    print(f"{'Problem':<20} | {'N':<4} | {'Best Score':<12} | {'Time (s)':<10} | Parameters")
    print("-" * 75)
    
    # Custom sorting key (using the base filename key stored in 'results')
    def custom_sort_key_summary(filename):
        parts = filename.split('_')
        prob_type = parts[1]
        size = int(parts[-1].replace('.npy', ''))
        return (prob_type, size)
    
    sorted_results = sorted(results.items(), key=lambda item: custom_sort_key_summary(item[0]))

    for filename, data in sorted_results:
        print(f"{filename:<20} | {data['N_cities']:<4} | {data['best_score']:<12} | {data['runtime_sec']:<10} | {data['parameters']}")
    
    print("-" * 75)


--- Starting Hybrid GA Execution (Speed Optimized) ---

Processing problem_g_1000.npy (N=1000). Params: Pop=100, Gen=1000
[problem_g_1000.npy] Final Score: 8838.44 | Time: 1202.21s

Processing problem_r1_1000.npy (N=1000). Params: Pop=100, Gen=1000
[problem_r1_1000.npy] Final Score: 3141.14 | Time: 1111.83s

Processing problem_r2_1000.npy (N=1000). Params: Pop=100, Gen=1000
[problem_r2_1000.npy] Final Score: 2618.75 | Time: 4801.71s


--- RESULTS SUMMARY (Hybrid GA: NumPy + 2-opt) ---
Problem              | N    | Best Score   | Time (s)   | Parameters
---------------------------------------------------------------------------
problem_g_1000.npy   | 1000 | 8838.44      | 1202.21    | Pop=100, Gen=1000, Mut=0.05, 2opt_F=300
problem_r1_1000.npy  | 1000 | 3141.14      | 1111.83    | Pop=100, Gen=1000, Mut=0.05, 2opt_F=300
problem_r2_1000.npy  | 1000 | 2618.75      | 4801.71    | Pop=100, Gen=1000, Mut=0.05, 2opt_F=300
----------------------------------------------------------------------